<a href="https://colab.research.google.com/github/mishiruv/EVA5/blob/master/EVA5_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
class Net(nn.Module):
    def __init__(self): # Initilising and defines the various layers in the CNN architecture
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)         #input-28*28*1,   kernelsize-3*3*1*32,    output-28*28*32,  RF-3*3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)        #input-28*28*32,  kernelsize-3*3*32*64,   output-28*28*64,  RF-5*5
        self.pool1 = nn.MaxPool2d(2, 2)                     #input-28*28*64,                          output-14*14*64,  RF-10*10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)       #input-14*14*64,  kernelsize-3*3*64*128,  output-14*14*128, RF-12*12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)      #input-14*14*128, kernelsize-3*3*128*256, output-14*14*256, RF-14*14
        self.pool2 = nn.MaxPool2d(2, 2)                     #input-14*14*256,                         output-7*7*256,   RF-28*28
        self.conv5 = nn.Conv2d(256, 512, 3)                 #input-7*7*256,   kernelsize-3*3*256*512, output-5*5*512,   RF-30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)                #input-5*5*512,   kernelsize-3*3*512*1024,output-3*3*1024,  RF-32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)                 #input-3*3*1024,  kernelsize-3*3*1024*10, Output-1*1*10

    def forward(self, x):                                         # Connecting the different layers in the CNN architecture and constructing a feed forward network
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # performs convolution1 -> relu activation -> convolution2 -> relu activation -> maxpool1
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # performs convolution3 -> relu activation -> convolution4 -> relu activation -> maxpool2
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             # performs convolution5 -> relu activation -> convolution6 -> relu activation
        x = F.relu(self.conv7(x))                                 # performs convolution7 
        x = x.view(-1, 10)                                        # Reshaping
        return F.log_softmax(x)                                   # perform a softmax along the 10 outputs

In [6]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()                 # check if nvidia cuda gpu is available
device = torch.device("cuda" if use_cuda else "cpu") # transfer the model to the device chosen above
model = Net().to(device)
summary(model, input_size=(1, 28, 28))                # print the summary of the model

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:
# Seed the model with value 1 to obtain consistent results
torch.manual_seed(1)

# Set the batch size to 128 (usually its 2^5)
batch_size = 128

# The below makes sure that the data stays in the memory
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# load the train data 
# Perform the standard normalization
# Normalisation shall do the following for each channel:
# image = (image - mean) / std
# The parameters mean, std are being passed

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# load the test data and perform standard normalization
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [8]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.7942557334899902 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 27.30it/s]



Test set: Average loss: 1.7246, Accuracy: 3798/10000 (38%)

